# Tarea 1

## Recuperación ranqueada y vectorización de documentos (RRDV) usando GENSIM

In [2]:
!pip install gensim

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/ad/97/b8253236dfedb9094f4273393a3fd03997da81f27f15822e56128da894ae/gensim-4.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.7.0 from https://files.pythonhosted.org/packages/06/15/e73734f9170b66c6a84a0bd7e03586e87e77404e2eb8e34749fc49fa43f7/scipy-1.11.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.1 kB ? eta -:--:--
     -------------------------- ----------- 41.0/59.1 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 59.1/59.1 kB 626.5 kB/s eta 0:00:00
     ---------------------------------------- 0.0/56.8 kB ? eta -:--:--
     ---------------------------------------- 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   - -------------------------------------- 1.1/24.0 MB

In [1]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import zipfile
from metrics import precision_at_k, recall_at_k, ndcg_at_k, mean_average_precision

from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
import nltk


In [2]:
# Se extraen los datos de los archivos comprimidos

# Specify the paths to the compressed files and the target directory
compressed_files = ['docs-raw-texts.zip', 'queries-raw-texts.zip']

# Extract files from each compressed file
for compressed_file in compressed_files:
    with zipfile.ZipFile(compressed_file, 'r') as zip_ref:
        folder_name = os.path.splitext(compressed_file)[0]  # Remove the ".zip" extension
        target_folder = os.path.join(folder_name)
        
        if not os.path.exists(target_folder):
            # Create the folder within the target directory
            os.mkdir(target_folder)

        
            # Extract all files to the target folder
            zip_ref.extractall(target_folder)

print("Extracción completada")

Extracción completada


In [3]:
# Extracts raw text from .naf files
def extract_raw_text(xml_path: str, title: bool = False) -> str:
    """Extrae el texto sin procesar de un archivo .naf.

    Args:
        xml_path (str): La ruta al archivo .naf.
        title (bool): Si es True, el título del documento también se agrega al texto extraído.

    Returns:
        Str: El texto sin procesar del archivo .naf.
    """
    if title:
        # Parse the XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()
        # Extract content from the XML
        return root.find(".//nafHeader/fileDesc").get("title") + ", " + root.find('raw').text #Añade título cuando se especifíca
    else:
        # Parse the XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()
        # Extract content from the XML
        return root.find('raw').text

# filtra los resultados de las listas para solo incluir los nombres de los documentos
def filter_result_list(input_list):
    return[full_result[:full_result.find(":")] for full_result in input_list]

In [4]:
# Load the relevance judgments from the file
relevance_judgments_file = open("relevance-judgments.tsv", "r")
relevance_judgments = {}

for line in relevance_judgments_file:
    query, judgments = line.strip().split('\t')
    relevance_judgments[query] = judgments.split(',')

relevance_judgments_file.close()

In [5]:
import gensim

# Download the NLTK stopwords resource
nltk.download('stopwords')

# Create a tokenizer
tokenizer = gensim.utils.simple_preprocess

# Create a stemmer
stemmer = gensim.parsing.preprocessing.PorterStemmer()

# Create a stopwords list
stopwords = nltk.corpus.stopwords.words('english')

# Define the preprocessing function
def preprocess_text(text: str) -> list[str]:
    """Preprocesa un texto para eliminar palabras vacías, aplicar stemming y convertir a minúsculas.

    Args:
        text (str): El texto a preprocesar.

    Returns:
        List: Una lista con las palabras del texto preprocesado.
    """

    # Convertir a minúsculas y eliminar espacios innecesarios
    text = text.strip().lower()

    # Tokenizar por espacio
    tokens = tokenizer(text)

    # Eliminar palabras vacías
    tokens = [token for token in tokens if token not in stopwords]

    # Aplicar stemming
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\57305\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Directorios que contienen los archivos necesarios, cambiar acá si es necesario
xml_files_directory = 'docs-raw-texts'

relevance_judgments_directory = "relevance-judgments.tsv"

queries_directory = "queries-raw-texts"

In [7]:
# Dictionary to store the inverted index (term -> list of documents)
inverted_index = {}

# Dictionary to store term frequencies per document (term -> {document: frequency})
term_freq_per_document = {}

# Iterate over XML files in the directory
for filename in os.listdir(xml_files_directory):
    if filename.endswith('.naf'):
        xml_path = os.path.join(xml_files_directory, filename)
        content = extract_raw_text(xml_path, title=True)
        # Preprocess the content
        preprocessed_tokens = preprocess_text(content)
        
        # Create the inverted index and update term frequencies per document
        for term in preprocessed_tokens:
            if term in inverted_index:
                if filename not in inverted_index[term]:
                    inverted_index[term].append(filename)
            else:
                inverted_index[term] = [filename]
            
            if term in term_freq_per_document:
                if filename in term_freq_per_document[term]:
                    term_freq_per_document[term][filename] += 1
                else:
                    term_freq_per_document[term][filename] = 1
            else:
                term_freq_per_document[term] = {filename: 1}

print("Inverted index created.")

Inverted index created.


In [11]:
import gensim

def preprocess_input(query_string):
    query_terms = gensim.utils.simple_preprocess(query_string)
    return {term: 1 for term in query_terms}

# Function to calculate TF-IDF vector for a query
def calculate_tfidf_vector(input, inverted_index, term_freq_per_document):
    total_documents = len(term_freq_per_document)
    query = preprocess_input(input)
    term_indices = {}  # Map term indices to integer indices in the tfidf_vector

    for term_index, term in enumerate(inverted_index):
        term_indices[term] = term_index

    tfidf_vector = np.zeros(len(inverted_index))  # Initialize a vector of zeros

    for term, term_index in inverted_index.items():
        tf = query.get(term, 0)  # Term frequency in the query
        df = len(term_index)

        tfidf = (np.log10(1 + tf)) * np.log10(total_documents / df)
        index = term_indices[term]
        tfidf_vector[index] = tfidf

    return tfidf_vector

# Example usage
query_string = "William Beaumont is Confused by human physiology"
tfidf_vector = calculate_tfidf_vector(query_string, inverted_index, term_freq_per_document)
print("TF-IDF Vector for query:", query_string)
print(tfidf_vector)
print(f"elements: {len(tfidf_vector)}")
print(f"non-zero elements:{sum(tfidf_vector>0)}")

TF-IDF Vector for query: William Beaumont is Confused by human physiology
[0.72338035 1.23482128 0.69954442 ... 0.         0.         0.        ]
elements: 12647
non-zero elements:3


a. Se preprocesa la consulta para eliminar stop words y se crea un diccionario que mapea cada término a su frecuencia en la consulta.

b. Se crea un vector de longitud igual al número de términos en el índice invertido. El vector se inicializa a 0.

c. Para cada término en el índice invertido, se calcula su TF-IDF. El TF-IDF se calcula como el producto de la frecuencia del término en la consulta y el logaritmo del número de documentos que contienen el término dividido por el número total de documentos.

d. El valor TF-IDF del término se almacena en la posición del término en el vector.

La estrategia utilizada es eficiente porque aprovecha la estructura del índice invertido. El índice invertido almacena una lista de documentos para cada término. Esto significa que podemos calcular la frecuencia de un término en una consulta en tiempo constante.

Además, el cálculo del TF-IDF también es eficiente. El TF-IDF se calcula como el producto de la frecuencia de un término en una consulta y el logaritmo del número de documentos que contienen el término. El logaritmo es una operación relativamente costosa, pero se puede realizar en tiempo constante utilizando una tabla de logaritmos precalculada.

In [ ]:
from gensim import matutils

def cosine_similarity(vector1, vector2):
    # Asegúrate de que los vectores tengan la misma longitud
    if len(vector1) != len(vector2):
        raise ValueError("Los vectores deben tener la misma longitud")
    
    # Calcula la similitud del coseno
    similarity = matutils.cossim(vector1, vector2)
    
    return similarity

In [ ]:
def retrieve_and_rank_documents(query_text):
    query_bow = inverted_index.doc2bow(query_text)
    query_tfidf = calculate_tfidf_vector[query_bow]
    sims = index[query_tfidf]
    
    ranked_documents = sorted(enumerate(sims), key=lambda item: -item[1])  # Ordenar por similitud en orden descendente
    ranked_documents = [(f'd{idx + 1:02}', score) for idx, score in ranked_documents if score > 0]  # Filtrar similitud > 0
    
    return ranked_documents

# Paso 4: Iterar sobre las consultas y escribir los resultados
output_filename = "GENSIM-consultas_resultados.tsv"

if os.path.exists(output_filename):
    print("Los resultados en {} ya existen.".format(output_filename))
else:
    # Abre el archivo de resultados en modo escritura
    with open(output_filename, "w") as results_file:
        # Itera sobre las consultas
        for i, query_text in enumerate(queries):
            ranked_documents = retrieve_and_rank_documents(query_text)
            
            # Escribir resultados en el archivo
            result_line = "q{:02d} ".format(i + 1) + ", ".join(["{}:{:.6f}".format(doc, score) for doc, score in ranked_documents])
            results_file.write(result_line + "\n")
            print("Consulta finalizada: q{:02d}".format(i + 1))

print("Los resultados se han escrito en " + output_filename)

